In [5]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [6]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [7]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PAYMENT_ID', 'des_col': 'payment_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONSUMER_ID', 'des_col': 'consumer_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACCOUNT_NUMBER', 'des_col': 'account_number', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OFFICE_ID', 'des_col': 'office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CATEGORY_ID', 'des_col': 'category_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BILL_ID', 'des_col': 'bill_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BILL_NO', 'des_col': 'bill_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BILL_DATE', 'des_col': 'bill_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'BILL_AMOUNT', 'des_col': 'bill_amount', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'AMOUNT_PAID', 'des_col': 'amount_paid', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'REBATE_ALLOWED', 'des_col': 'rebate_allowed', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PENALTY_ALLOWED', 'des_col': 'penalty_allowed', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'REBATE_PENDING', 'des_col': 'rebate_pending', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PENALTY_PENDING', 'des_col': 'penalty_pending', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PRNO', 'des_col': 'prno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REVERSE_TRANSACTION_DATE', 'des_col': 'reverse_transaction_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'REVERSE_TRANSACTION_BY', 'des_col': 'reverse_transaction_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REASON_FOR_MODIFICATION', 'des_col': 'reason_for_modification', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CANCELLED_STATUS', 'des_col': 'cancelled_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CANCELLED_BY', 'des_col': 'cancelled_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECONN_CHARGES', 'des_col': 'reconn_charges', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'ARREARS_AS_ON', 'des_col': 'arrears_as_on', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOURCE_OFFICE_ID', 'des_col': 'source_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TARGET_OFFICE_ID', 'des_col': 'target_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'FLAG1', 'des_col': 'flag1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG2', 'des_col': 'flag2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG3', 'des_col': 'flag3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG4', 'des_col': 'flag4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG5', 'des_col': 'flag5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MODULE_ID', 'des_col': 'module_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBMODULE_ID', 'des_col': 'submodule_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'POST_INDICATOR', 'des_col': 'post_indicator', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'POST_DATE', 'des_col': 'post_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'INSTALLMENT_ID', 'des_col': 'installment_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COINAGE_AMT', 'des_col': 'coinage_amt', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'ADVANCE_AMT', 'des_col': 'advance_amt', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PP_REF_NO', 'des_col': 'pp_ref_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PP_REF_DATE', 'des_col': 'pp_ref_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'PP_APPROVED_NAME', 'des_col': 'pp_approved_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PP_APPROVED_DESIG', 'des_col': 'pp_approved_desig', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LASTMONTH_REBATE', 'des_col': 'lastmonth_rebate', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'LASTMONTH_PENALTY', 'des_col': 'lastmonth_penalty', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'MANUAL_PRNO', 'des_col': 'manual_prno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'EMPLOYEE_CODE', 'des_col': 'employee_code', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISP_STATUS', 'des_col': 'disp_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'POSTED_DATE', 'des_col': 'posted_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'VERIFIED', 'des_col': 'verified', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'INITIATED', 'des_col': 'initiated', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUSPECTED', 'des_col': 'suspected', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COUNTER_CODE', 'des_col': 'counter_code', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PCCBNO', 'des_col': 'pccbno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'VGL_REG_ID', 'des_col': 'vgl_reg_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COLLECTION_DATE', 'des_col': 'collection_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'CURR_CAT', 'des_col': 'curr_cat', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CURR_PHASE', 'des_col': 'curr_phase', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CYCLE', 'des_col': 'cycle', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CYCLE_YEAR', 'des_col': 'cycle_year', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MAC_ADDRESS', 'des_col': 'mac_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DHARI_YEAR', 'des_col': 'dhari_year', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MOBILE_SERVICE_TRANS_ID', 'des_col': 'mobile_service_trans_id', 'src_dtype': 'String', 'des_dtype': 'String'}
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'CENTRAL_ALL.PAYMENT_DETAILS_T', 'public', 'payment_details_t', cols_map)
print("Total Migrated Records (payment_details_t): " + str(migrated_row_count))


Total Migrated Records (payment_details_t): Total Src Records: 201 Total migrated Records to dest: 203
